In [22]:
import os
import pandas as pd
import snscrape.modules.twitter as sntwitter
import numpy as np
import datetime
from capstone_twitter_search import twittsearch
import os
import pandas as pd

import numpy as np                               # linear algebra
import pandas as pd                              # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re                                        # to handle regular expressions
from string import punctuation                   # to extract the puntuation symbols
import nltk
from nltk.tokenize import word_tokenize          # to divide strings into tokens
from nltk.stem import WordNetLemmatizer          # to lemmatize the tokens
from nltk.corpus import stopwords                # to remove the stopwords 

import random                                    # for generating (pseudo-)random numbers
import matplotlib.pyplot as plt                  # to plot some visualizations
import datetime
import tensorflow as tf            
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import EarlyStopping
from mordecai import Geoparser
#from mordecai import batch_geoparse
from flatten_json import flatten_json
import numpy as np
import json
from tensorflow.keras import layers
import text_proccessing
from text_proccessing import clean_text
from text_proccessing import remove_stopwords
from text_proccessing import lemmatize_text
from text_proccessing import concatenate_text
from text_proccessing import makeglove
from text_proccessing import make_embedding_matrix
import append_df
from append_df import append_df
import modhelp
from modhelp import train_val_split
from modhelp import geo_df
from modhelp import suggest_nn2

In [8]:
code_dir = os.getcwd()
directory = os.path.join(data_dir,'splits')
parent_dir = os.path.dirname(code_dir)
data_dir = os.path.join(parent_dir,"Data")
dsa= os.path.dirname(parent_dir)
tweet_dir = os.path.join(parent_dir,"TweetMap")


In [ ]:
# #!/usr/bin/env python3
# # -*- coding: utf-8 -*-
# """
# Created on Mon Mar  1 10:40:16 2021

# @author: Arbo
# """

# def twittsearch(text_query,since_date,until_date,tweetcount):
#     tweets_list = []

#     query = '{0} since:{1} until:{2} filter:has_engagement'.format(text_query, since_date, until_date)
#     print(query)

#     # Using TwitterSearchScraper to scrape data and append tweets to list

#     for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):

#         if i>tweetcount:

#             break
#         tweets_list.append([tweet.date, tweet.id, tweet.content, 
#                             tweet.lang,tweet.retweetedTweet,tweet.quotedTweet])

#     # Creating a dataframe from the tweets list above

#     tweets_df = pd.DataFrame(tweets_list, 
#                              columns=['Datetime', 'TweetId', 'Text',
#                                      "Language", "RTFrom", "QTFrom"])
    
#     print("found {} tweets ranging from {} to {}".format(len(tweets_df),
#                                                             tweets_df.Datetime.min(),tweets_df.Datetime.max()))
    
#     print("dropping duplicates")   
#     tweets_df = tweets_df.drop_duplicates(subset=['TweetId'])
#     print("total of tweets now: {}".format(len(tweets_df)))
#     print("english only")

#     tweets_df = tweets_df[tweets_df["Language"]=='en']
#     print("total of tweets now: {}".format(len(tweets_df)))
#     tweets_df = tweets_df[tweets_df["RTFrom"].isna()]
#     print("total of tweets now: {}".format(len(tweets_df)))
#     tweets_df = tweets_df[tweets_df["QTFrom"].isna()]
#     print("total of tweets now: {} ranging from {} to {}".format(len(tweets_df),
#                                                                 tweets_df.Datetime.min(),tweets_df.Datetime.max()))
    
#     return tweets_df





In [ ]:
# scrapestart =  datetime.datetime.now() 
# print("Scraping commenced at {}".format(scrapestart))
# # =============================================================================
# text_query = '("forest fire") OR wildfire OR bushfire OR \
# (extreme heat) OR (record heat) OR heatwave OR ("heat wave") OR typhoon OR cyclone OR hurricane OR \
# tornado OR ("storm surge") OR  blizzard OR snow OR ("ice storm") OR sleet OR thunderstorm OR \
# hail OR flood OR flooding OR freeze OR frost OR (extreme cold) OR landslide OR tsunami OR ("tidal wave") OR \
# earthquake OR eruption OR volcano OR lava OR lahar OR avalanche OR mudslide OR sinkhole'


# # qlist = [firemetq_geologicalotherq]
# # qname= ['fire & meteorlogical & geological & other']
# since_date = '2021-07-06'
# until_date = '2021-07-07'
# tweetcount = 150000
# # tweets_df = twittsearch(qname,qlist,since_date,until_date,tweetcount)
# twittsearch(text_query,since_date,until_date,tweetcount).to_csv(os.path.join(tweet_dir,'tweets_df_{0}_{1}.csv'.
#                               format(since_date.replace('-',''),until_date.replace('-',''))),
#                  index =False)
# # =============================================================================
# scrapend =  datetime.datetime.now()
# print("Scraping ended at {}".format(scrapend))
# print("Scraping time {}".format(scrapend-scrapestart))

In [14]:
since_date = '2021-07-13'
until_date = '2021-07-14'

In [ ]:
# os.path.join(tweet_dir,'tweets_df_{0}_{1}.csv'.
#                               format(since_date.replace('-',''),until_date.replace('-','')))

In [6]:
train_data = pd.read_csv(os.path.join(directory, 'InformativenessTrain_Tokenized.csv'),engine='python')
train_data['text']= train_data['text'].astype(str)

In [9]:
start =  datetime.datetime.now() 
print('splitthis started at: {}'.format(start))
train_samples, val_samples, train_labels, val_labels = train_val_split(train_data, 0.25)
end =  datetime.datetime.now() 
print ('Text done {}'.format(end-start))
start =  datetime.datetime.now() 
print('Loading model,embedding and glove commenced at  {}'.format(start))

model = tf.keras.models.load_model(os.path.join(dsa,'model1'))
print("loading vecotor")
path_to_glove_file = os.path.join(dsa,'WordVector','glove.twitter.27B.200d.txt')

print("indexing")
embeddings_index=makeglove(path_to_glove_file)
print("matrix and vectorize")
embedding_matrix, vectorizer = make_embedding_matrix(train_samples, val_samples, embeddings_index)
end =  datetime.datetime.now() 

print("embedded in {}".format(end-start))

splitthis started at: 2021-07-15 17:06:24.118237
Total size of the dataset: 117339.
Training dataset: 88005.
Validation dataset: 29334.
Text done 0:00:00.304603
Loading model,embedding and glove commenced at  2021-07-15 17:06:24.423653
loading vecotor
indexing
matrix and vectorize
Converted 32239 words (22761 misses).
embedded in 0:03:51.935787


In [ ]:
# # result = pd.read_csv(os.path.join(tweet_dir,'result.csv'), engine='python')
# # result['Text'] = result['Text'].astype(str)
# # result_inf = result[result['target']==0]
# result_inf['Datetime']= pd.to_datetime(result_inf['Datetime'])

In [15]:
twts = pd.read_csv(os.path.join(tweet_dir,'tweets_df_{0}_{1}.csv'.
                              format(since_date.replace('-',''),until_date.replace('-',''))), engine = 'python')
twts['Text'] = twts['Text'].astype(str)


In [21]:
twts.sort_values(by='Datetime')

,Datetime,TweetId,Text,TweetLink,Likes,Replies,Retweets,QuoteTweets,Language,RTFrom,QTFrom
58780,2021-07-13 00:00:00+00:00,1414736329720561664,"During wildfire season, it's important to stay...",https://twitter.com/SJFD/status/14147363297205...,9,0,3,0,en,NaN,NaN
58779,2021-07-13 00:00:00+00:00,1414736329812938760,"rain, frost, gales: the bright light is cold, ...",https://twitter.com/notaleptic/status/14147363...,1,0,0,0,en,NaN,NaN
58778,2021-07-13 00:00:00+00:00,1414736331855613954,"If someone is missing a large plastic pipe, it...",https://twitter.com/WeatherNation/status/14147...,28,0,11,2,en,NaN,NaN
58774,2021-07-13 00:00:01+00:00,1414736336838434817,"People in Dexter, Missouri, are cleaning up th...",https://twitter.com/WPSDLocal6/status/14147363...,1,0,0,0,en,NaN,NaN
58777,2021-07-13 00:00:01+00:00,1414736333214568451,"Yes, aluminum foil can be used over windows to...",https://twitter.com/CBS8/status/14147363332145...,4,1,0,1,en,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4,2021-07-13 23:59:53+00:00,1415098689991831554,bleed it at mortality (29),https://twitter.com/90Frost/status/14150986899...,0,1,0,0,en,NaN,NaN
3,2021-07-13 23:59:54+00:00,1415098691862544409,When there’s a severe thunderstorm warning I f...,https://twitter.com/mj_inglis/status/141509869...,4,0,0,0,en,NaN,NaN
2,2021-07-13 23:59:54+00:00,1415098692886007809,"@Dallas_Holladay I don't see the problem, I li...",https://twitter.com/Fred_Chu/status/1415098692...,3,0,0,0,en,NaN,NaN
1,2021-07-13 23:59:55+00:00,1415098697273204737,THE NATIONAL WEATHER SERVICE IN POCATELLO HAS ...,https://twitter.com/MichaelCoatsWX/status/1415...,1,0,0,0,en,NaN,NaN


In [23]:
procstart =  datetime.datetime.now()
print("{}: processing ".format(procstart))
twts['ptext'] = twts['Text'].apply(lambda x: clean_text(x))
twts['ptext'] = twts['ptext'].apply(lambda x: word_tokenize(x))
twts['ptext'] = twts['ptext'].apply(lambda x : remove_stopwords(x))
twts['ptext'] = twts['ptext'].apply(lambda x : lemmatize_text(x))
twts['ptext'] = twts['ptext'].apply(lambda x : concatenate_text(x))
procend=  datetime.datetime.now() 
print("{}: processed in {}".format(procend, procend - procstart))



2021-07-15 17:17:12.837638: processing 
2021-07-15 17:20:43.449239: processed in 0:03:30.611601


In [24]:

predstart = datetime.datetime.now() 
print("{}: predict".format(predstart))
predictions = suggest_nn2(twts, model,vectorizer)

submission_data = {"ID": twts['TweetId'].tolist(),"tweet": twts['Text'].tolist(), "target": predictions}

submission_df = pd.DataFrame(submission_data)
result = twts.join(submission_df.target)
predend = datetime.datetime.now() 
print("predicted {}".format(predend-predstart))
# 
result_inf = result[result['target']==0]
print("working over {} informative tweets".format(len(result_inf)))



2021-07-15 17:23:55.253515: predict
predicted 0:01:30.184650
working over 40821 informative tweets


In [ ]:


print("Run geoprocessing over tweets")

geostart =  datetime.datetime.now() 
print("{}: geostart".format(geostart))
df_js = geo_df(result_inf)
geoend =  datetime.datetime.now()
print("{}: geoend {}".format(geoend,geoend-geostart))

df_js.to_csv(os.path.join(tweet_dir,'result_{0}_{1}.csv'.
                              format(since_date.replace('-',''),until_date.replace('-',''))))
print("fín")

Run geoprocessing over tweets
2021-07-15 17:29:21.311779: geostart
Models path: /Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/mordecai/models/
You may be using an outdated Geonames index/Mordecai version.
Your index is from 2018-06-05, while your Mordecai version is from 2020-07-11. Please see
https://github.com/openeventdata/mordecai/ for instructions on updating.


  0%|          | 126/40821 [00:15<2:13:43,  5.07it/s]Traceback (most recent call last):
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1629, in predict
    tmp_batch_outputs = self.predict_function(iterator)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 862, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning

None
None


  1%|          | 252/40821 [00:25<46:12, 14.63it/s]  Traceback (most recent call last):
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1629, in predict
    tmp_batch_outputs = self.predict_function(iterator)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 862, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning

None


...

  8%|▊         | 3404/40821 [04:30<1:29:37,  6.96it/s]Traceback (most recent call last):
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1629, in predict
    tmp_batch_outputs = self.predict_function(iterator)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 862, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearnin

None


  8%|▊         | 3441/40821 [04:31<24:06, 25.84it/s]Traceback (most recent call last):
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1629, in predict
    tmp_batch_outputs = self.predict_function(iterator)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 862, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/

None


  9%|▉         | 3765/40821 [04:51<33:07, 18.65it/s]  

In [ ]:
t2 = tweets_df[tweets_df.QTFrom.isna() ]
t2.QTFrom.unique()


In [ ]:
# tweets_df.to_csv(os.path.join(tweet_dir,'tweets_df_{0}_{1}.csv'.
#                               format(since_date.replace('-',''),until_date.replace('-',''))),
#                  index =False)

In [ ]:
tweets_df